# Handling Missing Values

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('../Dataset/GUIDE_Train.csv')  

In [2]:
# Identify missing values
print("Missing Values:\n", df.isnull().sum())

Missing Values:
 Id                          0
OrgId                       0
IncidentId                  0
AlertId                     0
Timestamp                   0
DetectorId                  0
AlertTitle                  0
Category                    0
MitreTechniques       5468386
IncidentGrade           51340
ActionGrouped         9460773
ActionGranular        9460773
EntityType                  0
EvidenceRole                0
DeviceId                    0
Sha256                      0
IpAddress                   0
Url                         0
AccountSid                  0
AccountUpn                  0
AccountObjectId             0
AccountName                 0
DeviceName                  0
NetworkMessageId            0
EmailClusterId        9420025
RegistryKey                 0
RegistryValueName           0
RegistryValueData           0
ApplicationId               0
ApplicationName             0
OAuthApplicationId          0
ThreatFamily          9441956
FileName               

In [3]:
missing_threshold = 0.5  # set the threshold for missing values

# 1. Remove columns with more than 50% of missing values
missing_percentage = df.isnull().mean()
columns_to_drop = missing_percentage[missing_percentage > missing_threshold].index
df.drop(columns=columns_to_drop, inplace=True)

In [4]:
df.isnull().sum()

Id                        0
OrgId                     0
IncidentId                0
AlertId                   0
Timestamp                 0
DetectorId                0
AlertTitle                0
Category                  0
IncidentGrade         51340
EntityType                0
EvidenceRole              0
DeviceId                  0
Sha256                    0
IpAddress                 0
Url                       0
AccountSid                0
AccountUpn                0
AccountObjectId           0
AccountName               0
DeviceName                0
NetworkMessageId          0
RegistryKey               0
RegistryValueName         0
RegistryValueData         0
ApplicationId             0
ApplicationName           0
OAuthApplicationId        0
FileName                  0
FolderPath                0
ResourceIdName            0
OSFamily                  0
OSVersion                 0
CountryCode               0
State                     0
City                      0
dtype: int64

In [5]:
# fill missing values with the mode of the column
df['IncidentGrade'].fillna(df['IncidentGrade'].mode()[0], inplace=True)

C:\Users\aviji\AppData\Local\Temp\ipykernel_12056\3449365735.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['IncidentGrade'].fillna(df['IncidentGrade'].mode()[0], inplace=True)


In [6]:
# remove duplicates
df.drop_duplicates(inplace=True)    

# Removing Outliers

In [7]:
from scipy.stats import zscore    # importing the zscore function
import numpy as np                # importing the numpy library 

# function to remove outliers
def remove_outliers(df, threshold=3):
    numerical_df = df.select_dtypes(include=[np.number])    # Select only numerical columns
    z_scores = np.abs((numerical_df - numerical_df.mean()) / numerical_df.std())    # calculating the zscore

    # Filter out rows with Z-scores above the threshold in any column
    df_clean = df[(z_scores < threshold).all(axis=1)].copy()
    return df_clean     # returning the dataframe

print("Before removing outliers:", df.shape)    # printing the shape of the dataframe

# Remove outliers using Z-score
df = remove_outliers(df)  

print("After removing outliers:", df.shape)             # printing the shape of the dataframe

Before removing outliers: (8970539, 35)
After removing outliers: (4910238, 35)


# Feature Engineering

In [8]:
# Separate categorical and numerical columns
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns.remove('IncidentGrade')

In [9]:
constant_features = [col for col in numerical_columns if df[col].nunique() == 1]    # finding the constant features
print("Constant Features:", constant_features)
df.drop(columns=constant_features, inplace=True)    # dropping the constant features
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

Constant Features: ['RegistryValueName', 'RegistryValueData']


In [10]:
from sklearn.feature_selection import f_classif, chi2

target = 'IncidentGrade'    # target column

# Perform ANOVA test (f_classif)
anova_scores, p_values = f_classif(df[numerical_columns], df[target])
significance_level = 0.05   # Set significance threshold for feature selection
numerical_to_keep = [num for num, p in zip(numerical_columns, p_values) if p < significance_level]
numerical_to_drop = [num for num, p in zip(numerical_columns, p_values) if p >= significance_level]

print(f"Numerical Features Kept: {numerical_to_keep}")
print(f"Numerical Features Dropped: {numerical_to_drop}")

Numerical Features Kept: ['Id', 'OrgId', 'IncidentId', 'AlertId', 'DetectorId', 'AlertTitle', 'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId', 'ApplicationId', 'ApplicationName', 'FileName', 'FolderPath', 'ResourceIdName', 'CountryCode', 'State', 'City']
Numerical Features Dropped: ['RegistryKey', 'OAuthApplicationId', 'OSFamily', 'OSVersion']


In [11]:
df.drop(columns=numerical_to_drop, inplace=True)    # dropping unnecessary numerical features

In [12]:
from scipy.stats import chi2_contingency

significance_level = 0.05   # setting the significance level
categorical_to_keep = []
categorical_to_drop = []

for col in categorical_columns:
    # Create contingency table
    contingency_table = pd.crosstab(df[col], df[target])
    chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)     # performing the chi2 test
    
    # Keep or drop feature based on p-value
    if p_value < significance_level:
        categorical_to_keep.append(col)
    else:
        categorical_to_drop.append(col)

print(f"Categorical Features Kept: {categorical_to_keep}")
print(f"Categorical Features Dropped: {categorical_to_drop}")

Categorical Features Kept: ['Timestamp', 'Category', 'EntityType', 'EvidenceRole']
Categorical Features Dropped: []


# Encoding Catagorical Variables

In [13]:
df[categorical_columns].nunique()   # checking the unique values in the categorical columns

Timestamp       665569
Category            18
EntityType          26
EvidenceRole         2
dtype: int64

In [14]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])   # converting the timestamp column to datetime format

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Label Encoding
le = LabelEncoder()
df['Timestamp'] = le.fit_transform(df['Timestamp'])
df['EvidenceRole'] = le.fit_transform(df['EvidenceRole'])
df['IncidentGrade'] = le.fit_transform(df['IncidentGrade'])
df['Category'] = le.fit_transform(df['Category'])   
df['EntityType'] = le.fit_transform(df['EntityType'])

In [16]:
df.to_csv('cleaned_data.csv', index=False)    # saving the cleaned data to a csv file